In [ ]:
#librerías y paquetes

import numpy as np 
import tensorflow as tf
import pandas as pd
import keras_tuner as tk # instalar por cmd
from sklearn.model_selection import train_test_split
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.utils import to_categorical
from matplotlib import pyplot


In [ ]:
# Cargar base de datos

df = pd.read_csv(r"C:\Users\carlo\OneDrive\Escritorio\JAVERIANA\Clasificación RN _HP\BASE DE DATOS.csv", delimiter=';', decimal=',')  
df.info()
print(df)

In [ ]:
# Separar variables de entrada y objetivo

y = df['CLAS']
X = df.drop(columns = 'CLAS')

print(y)
print(X)


In [ ]:
# Escalar datos de entrada
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

XS = scaler.fit_transform(X)

print(XS)
print(XS.shape)





In [ ]:
# Categorizar las clases
o = pd.get_dummies(y, columns = ['CLAS'])
print(o)

In [ ]:
#Dividir en entrenamiento y prueba

# Definimos la semilla para el generador de número aleatorios
np.random.seed(7926)

# Dividimos los datos aleatoriamente en 80% para entrenamiento y 20% para prueba 
X_train, X_test, y_train, y_test = train_test_split(XS,o,test_size = 0.20, stratify=o)


# Chequeemos los resultados
print(X_train.shape)
print(y_train.shape)

print(X_test.shape)
print(y_test.shape)

In [ ]:
# Definir el modelo
model = Sequential()
model.add(Dense(50, input_dim=4, activation='relu',use_bias=False, kernel_initializer='he_uniform'))
model.add(Dense(10, activation='linear',use_bias=False, kernel_initializer='he_uniform'))
model.add(Dense(3, activation='softmax',use_bias=False))

In [ ]:

opt = Adam(learning_rate=0.08)
model.compile(loss='mean_squared_error', optimizer=opt, metrics=['accuracy'])
model.fit(X_train,y_train,epochs=100)
print(model.summary())

In [ ]:
Xnew = [[0.38,0.02,1,1]]
print(Xnew)

ynew = model.predict(Xnew)
print(ynew)

In [ ]:
Xnew = X_test
ynew = model.predict(X_test)
print(ynew)

In [ ]:
# Función para probar hiperparámetros

def model_builder(hp):
    
  hp_activation = hp.Choice('activation', values=['relu', 'tanh','linear'])
  hp_layer_1 = hp.Int('layer_1', min_value=1, max_value=1000, step=100)
  hp_layer_2 = hp.Int('layer_2', min_value=1, max_value=1000, step=100)
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
 
  model = Sequential()
  model.add(tf.keras.layers.Dense(units=hp_layer_1, activation=hp_activation))
  model.add(tf.keras.layers.Dense(units=hp_layer_2, activation=hp_activation))
  model.add(Dense(3, activation='softmax',use_bias=False))

  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),loss='mean_squared_error',metrics=['accuracy'])
  
  return model

In [ ]:
# Algoritmo Hyperband

tuner = tk.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=100,
                     factor=5,directory='/notebooks/Documents/NOTEBOOKS/RNCLASS-HP6',
                     project_name='RNCLASS-HP6')

In [ ]:
# crieterio de parada anticipada
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [ ]:
# Búsqueda
tuner.search(X_train, y_train, epochs=50, validation_data=(X_test, y_test), callbacks=[stop_early],verbose=True)
best_model = tuner.get_best_models()[0]


In [ ]:
#Resúmen
tuner.search_space_summary()


In [ ]:
# Resúmen mejores parámetros
tuner.results_summary()

In [ ]:
model1 = Sequential()
   
model1.add(tf.keras.layers.Dense(units=, activation=""))
model1.add(tf.keras.layers.Dense(units=601, activation=""))
model1.add(Dense(3, activation='softmax',use_bias=False))

model1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=),loss='mean_squared_error',metrics=['accuracy'])
model1.fit(X_train,y_train,epochs=100)
print(model1.summary())